In [2]:
import utils.losses as loss
import datasets.datasets as dtset

In [3]:
#Data = dtset.OpenML_Dataset('iris')
Data = dtset.Torch_Dataset('mnist')

In [4]:
Data.

'MNIST'

In [85]:
train_X,train_y,test_X,test_y =  Data.get_data()
print(train_X.shape)

torch.Size([105, 4])


In [86]:
print(train_X[:5,:])
print(train_y[:5,:])

tensor([[5.6000, 2.8000, 4.9000, 2.0000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [4.4000, 2.9000, 1.4000, 0.2000]])
tensor([[0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]])


In [87]:
import torch
import torch.nn as nn

# Define the MLP class
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    # Define layers
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.softmax = nn.Softmax(dim=1)  # For classification tasks

  def forward(self, x):
    # Forward pass through the layers
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

model = MLP(4, 2, 3)

prediction = model(train_X)
print(prediction[:5,:])


tensor([[-0.6728,  0.1304,  0.5010],
        [-0.6728,  0.1304,  0.5010],
        [-0.6728,  0.1304,  0.5010],
        [-0.6728,  0.1304,  0.5010],
        [-0.6728,  0.1304,  0.5010]], grad_fn=<SliceBackward0>)


In [88]:
train_dataloader,test_dataloader =  Data.get_dataloader()

In [89]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#EM = True
#criterion = loss.EMLoss(torch.eye(3))
EM = False
#criterion = loss.OSLCELoss()
criterion = loss.OSLBrierLoss()


In [90]:
# Training loop (with improvements):
for epoch in range(101):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # Get the inputs; forward pass:
        inputs, labels = data
        #print(torch.max(labels,1).indices)
        outputs = model(inputs)
        if EM == True:
            loss = criterion(outputs,(torch.max(labels,1).indices))
        else:
            loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(running_loss)
    
print('Finished training!')

37.35705089569092
37.00782871246338
36.090139389038086
35.46928024291992
34.77353477478027
33.979498863220215
33.11699295043945
32.20557689666748
31.25935935974121
30.29146957397461
29.316728591918945
28.354249954223633
27.450151443481445
26.608169555664062
25.823309898376465
25.119680404663086
24.508974075317383
23.97327995300293
23.503533363342285
23.087461471557617
22.72182559967041
22.39901638031006
22.11386489868164
21.858572959899902
21.63170623779297
21.43064785003662
21.25201416015625
21.090481758117676
20.943129539489746
20.807109832763672
20.683472633361816
20.569812774658203
20.464648246765137
20.36759853363037
20.27753257751465
20.194068908691406
20.1168155670166
20.04537296295166
19.979132652282715
19.91779136657715
19.860902786254883
19.807905197143555
19.758337020874023
19.711816787719727
19.667977333068848
19.626835823059082
19.588027954101562
19.551359176635742
19.516657829284668
19.483769416809082
19.452558517456055
19.422897338867188
19.394676208496094
19.36778831481

In [91]:
for images, labels in test_dataloader:
    outputs = model(images)
    outputs
    print(torch.max(outputs.data, 1))

torch.return_types.max(
values=tensor([0.6144, 2.7397, 0.3232, 0.6144, 0.6144, 2.1129, 0.6144, 2.5614, 0.6144,
        1.9886, 2.2009, 2.5325, 0.6144, 0.6144, 0.3179, 0.5732, 0.5055, 2.5647,
        2.7056, 1.8507, 0.6144, 2.5071, 0.2641, 2.5479, 0.5399, 0.6144, 2.1435,
        0.6144, 1.9751, 0.6144, 2.5665, 2.2811, 2.0809, 0.6144, 0.5141, 2.4033,
        0.6144, 0.6144, 2.3574, 0.3067, 0.5380, 0.6144, 0.3923, 2.5497, 0.3346]),
indices=tensor([1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]))


In [92]:

# Evaluate the model:
total_correct = 0
total_data = 0
with torch.no_grad():  # Disable gradient calculation for testing
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class label
        print(predicted)
        print(torch.max(labels,1)[1])
        total_correct += (predicted == torch.max(labels,1)[1]).sum().item()  # Count correct predictions
        total_data += labels.size(0)

accuracy = total_correct / total_data
print('Accuracy of the network on the test images: %d %%' % (100 * accuracy))


tensor([1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1])
tensor([2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0,
        1, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 1, 0, 1])
Accuracy of the network on the test images: 66 %
